In [1]:
import os
from pathlib import Path
from tempfile import TemporaryDirectory

from traitlets.utils.bunch import Bunch

from IPython.utils.capture import capture_output

from pylint.config import ConfigurationMixIn
from pylint.pyreverse.inspector import Linker, project_from_files
from pylint.pyreverse.diadefslib import DiadefsHandler
from pylint.pyreverse import writer
from pylint.pyreverse.utils import insert_default_options
from pylint.pyreverse.writer import DotWriter
from pylint.graph import DotBackend
from base64 import b64encode
from urllib.parse import quote

In [2]:
import pydot
import io
import pygraphviz as pgv
from IPython.display import Image, SVG, HTML, Markdown

In [3]:
import ipylintotype
from ipylintotype.diagnosers.diagnoser import IPythonDiagnoser
%reload_ext ipylintotype

In [4]:
ipl = ipylintotype.get_ipylintotype()

In [5]:
def pyreverse(code_str, **_config):
    with TemporaryDirectory() as td:
        tdp = Path(td)
        source = tdp / "source.py"
        source.write_text(code_str)
        default_config = dict(
            module_names="source",
            classes=[],
            mode="PUB_ONLY",  # or "ALL"
            all_ancestors=True,
            all_associated=True,
            show_ancestors=True,
            show_associated=True,
            only_classnames=True,
            output_format="dot",
            show_builtin=False,
        )

        config = dict()
        config.update(default_config)
        config.update(_config)

        config_bunch = Bunch(config)

        with capture_output(display=False):
            project = project_from_files([str(source)])
        linker = Linker(project, tag=True)
        handler = DiadefsHandler(config_bunch)
        diadefs = handler.get_diadefs(project, linker)
        old_cwd = os.getcwd()
        os.chdir(td)
        try:
            writer.DotWriter(config_bunch).write(diadefs)
        finally:
            os.chdir(old_cwd)
        for path in sorted(Path(td).glob("*.dot")):
            yield path.read_text().replace(f"{td}/source.py.", "")

In [6]:
graphs = list(
    pyreverse(
        """
from traitlets import HasTraits, Instance
class A(HasTraits):
    pass
class B(A):
    a = Instance(A)
class M:
    pass
class C(B, M):
    pass
"""
    )
)

In [7]:
print(graphs[0])

digraph "classes_no_name" {
charset="utf-8"
rankdir=BT
"0" [label="A", shape="record"];
"1" [label="B", shape="record"];
"2" [label="C", shape="record"];
"3" [label="M", shape="record"];
"4" [label="traitlets.traitlets.ClassBasedTraitType", shape="record"];
"5" [label="traitlets.traitlets.HasTraits", shape="record"];
"6" [label="traitlets.traitlets.Instance", shape="record"];
"0" -> "5" [arrowhead="empty", arrowtail="none"];
"1" -> "0" [arrowhead="empty", arrowtail="none"];
"2" -> "1" [arrowhead="empty", arrowtail="none"];
"2" -> "3" [arrowhead="empty", arrowtail="none"];
"6" -> "4" [arrowhead="empty", arrowtail="none"];
"6" -> "1" [arrowhead="diamond", arrowtail="none", fontcolor="green", label="a", style="solid"];
}



In [8]:
dot = pgv.AGraph(graphs[0])
# Image(data=dot.create("dot", "png"))
s = io.BytesIO()
dot.draw(s, "svg", prog="dot")

In [9]:
Markdown(f"""![svg image](data:image/svg+xml,{quote(s.getvalue())})""")

![svg image](data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%0A%3C%21DOCTYPE%20svg%20PUBLIC%20%22-//W3C//DTD%20SVG%201.1//EN%22%0A%20%22http%3A//www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd%22%3E%0A%3C%21--%20Generated%20by%20graphviz%20version%202.38.0%20%2820140413.2041%29%0A%20--%3E%0A%3C%21--%20Title%3A%20classes_no_name%20Pages%3A%201%20--%3E%0A%3Csvg%20width%3D%22453pt%22%20height%3D%22281pt%22%0A%20viewBox%3D%220.00%200.00%20453.00%20281.00%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%20xmlns%3Axlink%3D%22http%3A//www.w3.org/1999/xlink%22%3E%0A%3Cg%20id%3D%22graph0%22%20class%3D%22graph%22%20transform%3D%22scale%281%201%29%20rotate%280%29%20translate%284%20277%29%22%3E%0A%3Ctitle%3Eclasses_no_name%3C/title%3E%0A%3Cpolygon%20fill%3D%22white%22%20stroke%3D%22none%22%20points%3D%22-4%2C4%20-4%2C-277%20449%2C-277%20449%2C4%20-4%2C4%22/%3E%0A%3C%21--%200%20--%3E%0A%3Cg%20id%3D%22node1%22%20class%3D%22node%22%3E%3Ctitle%3E0%3C/title%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2279%2C-162.5%2079%2C-198.5%20133%2C-198.5%20133%2C-162.5%2079%2C-162.5%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22106%22%20y%3D%22-176.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3EA%3C/text%3E%0A%3C/g%3E%0A%3C%21--%205%20--%3E%0A%3Cg%20id%3D%22node2%22%20class%3D%22node%22%3E%3Ctitle%3E5%3C/title%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%220%2C-236.5%200%2C-272.5%20212%2C-272.5%20212%2C-236.5%200%2C-236.5%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22106%22%20y%3D%22-250.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Etraitlets.traitlets.HasTraits%3C/text%3E%0A%3C/g%3E%0A%3C%21--%200%26%2345%3B%26gt%3B5%20--%3E%0A%3Cg%20id%3D%22edge1%22%20class%3D%22edge%22%3E%3Ctitle%3E0%26%2345%3B%26gt%3B5%3C/title%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M106%2C-198.563C106%2C-206.693%20106%2C-216.624%20106%2C-225.795%22/%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22102.5%2C-226.059%20106%2C-236.059%20109.5%2C-226.059%20102.5%2C-226.059%22/%3E%0A%3C/g%3E%0A%3C%21--%201%20--%3E%0A%3Cg%20id%3D%22node3%22%20class%3D%22node%22%3E%3Ctitle%3E1%3C/title%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2279%2C-88.5%2079%2C-124.5%20133%2C-124.5%20133%2C-88.5%2079%2C-88.5%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22106%22%20y%3D%22-102.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3EB%3C/text%3E%0A%3C/g%3E%0A%3C%21--%201%26%2345%3B%26gt%3B0%20--%3E%0A%3Cg%20id%3D%22edge2%22%20class%3D%22edge%22%3E%3Ctitle%3E1%26%2345%3B%26gt%3B0%3C/title%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M106%2C-124.563C106%2C-132.693%20106%2C-142.624%20106%2C-151.795%22/%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22102.5%2C-152.059%20106%2C-162.059%20109.5%2C-152.059%20102.5%2C-152.059%22/%3E%0A%3C/g%3E%0A%3C%21--%202%20--%3E%0A%3Cg%20id%3D%22node4%22%20class%3D%22node%22%3E%3Ctitle%3E2%3C/title%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2243%2C-0.5%2043%2C-36.5%2097%2C-36.5%2097%2C-0.5%2043%2C-0.5%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2270%22%20y%3D%22-14.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3EC%3C/text%3E%0A%3C/g%3E%0A%3C%21--%202%26%2345%3B%26gt%3B1%20--%3E%0A%3Cg%20id%3D%22edge3%22%20class%3D%22edge%22%3E%3Ctitle%3E2%26%2345%3B%26gt%3B1%3C/title%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M77.2851%2C-36.9034C82.3442%2C-48.9888%2089.1786%2C-65.3156%2094.9107%2C-79.0089%22/%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2291.7586%2C-80.5431%2098.8486%2C-88.416%2098.2157%2C-77.8401%2091.7586%2C-80.5431%22/%3E%0A%3C/g%3E%0A%3C%21--%203%20--%3E%0A%3Cg%20id%3D%22node5%22%20class%3D%22node%22%3E%3Ctitle%3E3%3C/title%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%227%2C-88.5%207%2C-124.5%2061%2C-124.5%2061%2C-88.5%207%2C-88.5%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2234%22%20y%3D%22-102.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3EM%3C/text%3E%0A%3C/g%3E%0A%3C%21--%202%26%2345%3B%26gt%3B3%20--%3E%0A%3Cg%20id%3D%22edge4%22%20class%3D%22edge%22%3E%3Ctitle%3E2%26%2345%3B%26gt%3B3%3C/title%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M62.7149%2C-36.9034C57.6558%2C-48.9888%2050.8214%2C-65.3156%2045.0893%2C-79.0089%22/%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2241.7843%2C-77.8401%2041.1514%2C-88.416%2048.2414%2C-80.5431%2041.7843%2C-77.8401%22/%3E%0A%3C/g%3E%0A%3C%21--%204%20--%3E%0A%3Cg%20id%3D%22node6%22%20class%3D%22node%22%3E%3Ctitle%3E4%3C/title%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22151%2C-88.5%20151%2C-124.5%20445%2C-124.5%20445%2C-88.5%20151%2C-88.5%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22298%22%20y%3D%22-102.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Etraitlets.traitlets.ClassBasedTraitType%3C/text%3E%0A%3C/g%3E%0A%3C%21--%206%20--%3E%0A%3Cg%20id%3D%22node7%22%20class%3D%22node%22%3E%3Ctitle%3E6%3C/title%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22156.5%2C-0.5%20156.5%2C-36.5%20359.5%2C-36.5%20359.5%2C-0.5%20156.5%2C-0.5%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22258%22%20y%3D%22-14.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Etraitlets.traitlets.Instance%3C/text%3E%0A%3C/g%3E%0A%3C%21--%206%26%2345%3B%26gt%3B1%20--%3E%0A%3Cg%20id%3D%22edge5%22%20class%3D%22edge%22%3E%3Ctitle%3E6%26%2345%3B%26gt%3B1%3C/title%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M227.606%2C-36.6967C203.424%2C-50.3787%20169.466%2C-69.5915%20143.579%2C-84.2384%22/%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22143.474%2C-84.2974%20140.222%2C-90.7334%20133.03%2C-90.2066%20136.283%2C-83.7706%20143.474%2C-84.2974%22/%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22196.5%22%20y%3D%22-58.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22green%22%3Ea%3C/text%3E%0A%3C/g%3E%0A%3C%21--%206%26%2345%3B%26gt%3B4%20--%3E%0A%3Cg%20id%3D%22edge6%22%20class%3D%22edge%22%3E%3Ctitle%3E6%26%2345%3B%26gt%3B4%3C/title%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M266.095%2C-36.9034C271.716%2C-48.9888%20279.31%2C-65.3156%20285.679%2C-79.0089%22/%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22282.663%2C-80.8249%20290.054%2C-88.416%20289.01%2C-77.8727%20282.663%2C-80.8249%22/%3E%0A%3C/g%3E%0A%3C/g%3E%0A%3C/svg%3E%0A)

In [10]:
import typing as typ
from ipylintotype import shapes
from IPython import InteractiveShell

class PyReverseDiagnoser(IPythonDiagnoser): 
    
    def run(
        self,
        cell_id: typ.Text,
        code: typ.List[shapes.Cell],
        metadata: shapes.Metadata,
        shell: InteractiveShell,
        *args,
        **kwargs
    ) -> shapes.Annotations:
        transformed_code, line_offsets = self.transform_for_diagnostics(code, shell)
        graphs = list(pyreverse(transformed_code))
        if not graphs:
            return {}

        dot = pgv.AGraph(graphs[0])
        s = io.BytesIO()
        dot.draw(s, "svg", prog="dot")
        md = f"""![svg image](data:image/svg+xml,{quote(s.getvalue())})"""
        
        return dict(
            markup_contexts=[
                {
                    "title": f"Class Diagram",
                    "range": {
                        "start": dict(line=0, character=0),
                        "end": dict(line=0, character=1),
                    },
                    "content": {
                        "kind": "markdown",
                        "value": md,
                    }
                }
            ]
        )
        return {}

In [11]:
ipl.diagnosers = [PyReverseDiagnoser()]